In [8]:
image_size = 224
num_channels = 3
num_classes = 120
project_path ='Documents/OC_Ingenieur_ML/P6_Classez_des_images/'

from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras import Model,callbacks
# Charger VGG-16 pré-entraîné sur ImageNet et sans les couches fully-connected
model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))

# Récupérer la sortie de ce réseau
x = model.layers[-2].output

# Ajouter la nouvelle couche fully-connected pour la classification à 10 classes

predictions = Dense(num_classes, activation='softmax')(x)

# Définir le nouveau modèle
new_model = Model(inputs=model.input, outputs=predictions)

# Figer tous les poids
for layer in new_model.layers[:]:
    layer.trainable = False

In [9]:
import tensorflow as tf
%cd
# Compiler le modèle
new_model.compile(loss="categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  metrics=["accuracy"])

new_model.load_weights(project_path+'models/model120_weights.h5')
new_model.save(project_path+'models/model120.h5')
new_model.summary()

/home/fedecabre
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56,

In [40]:
from keras.preprocessing.image import ImageDataGenerator
import pickle

crop_images_folders_path = project_path+"data/Cropped/"

datagen = ImageDataGenerator()

generator = datagen.flow_from_directory(crop_images_folders_path,
                                                target_size=(image_size, image_size),
                                                batch_size=32,
                                                subset="training",
                                                class_mode='categorical')

num_classes = train_generator.num_classes
train_size = train_generator.n
races = generator.class_indices.keys()
races_dict = generator.class_indices
races_invert_dict = {v: k for k, v in races_dict.items()}

races_invert_dict_file = open(project_path+"models/races_invert_dict.pkl", "wb")
pickle.dump(races_invert_dict, races_invert_dict_file)
races_invert_dict_file.close()

races_invert_dict_file = open(project_path+"models/races_invert_dict.pkl", "rb")
races_invert_dict = pickle.load(races_invert_dict_file)
races_invert_dict

Found 20579 images belonging to 120 classes.


{0: 'n02085620-Chihuahua',
 1: 'n02085782-Japanese_spaniel',
 2: 'n02085936-Maltese_dog',
 3: 'n02086079-Pekinese',
 4: 'n02086240-Shih-Tzu',
 5: 'n02086646-Blenheim_spaniel',
 6: 'n02086910-papillon',
 7: 'n02087046-toy_terrier',
 8: 'n02087394-Rhodesian_ridgeback',
 9: 'n02088094-Afghan_hound',
 10: 'n02088238-basset',
 11: 'n02088364-beagle',
 12: 'n02088466-bloodhound',
 13: 'n02088632-bluetick',
 14: 'n02089078-black-and-tan_coonhound',
 15: 'n02089867-Walker_hound',
 16: 'n02089973-English_foxhound',
 17: 'n02090379-redbone',
 18: 'n02090622-borzoi',
 19: 'n02090721-Irish_wolfhound',
 20: 'n02091032-Italian_greyhound',
 21: 'n02091134-whippet',
 22: 'n02091244-Ibizan_hound',
 23: 'n02091467-Norwegian_elkhound',
 24: 'n02091635-otterhound',
 25: 'n02091831-Saluki',
 26: 'n02092002-Scottish_deerhound',
 27: 'n02092339-Weimaraner',
 28: 'n02093256-Staffordshire_bullterrier',
 29: 'n02093428-American_Staffordshire_terrier',
 30: 'n02093647-Bedlington_terrier',
 31: 'n02093754-Border_

In [43]:
import cv2

image_path = project_path+'data/COCO/coco.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
image = image.reshape(1, 224, 224, 3)
prediction = new_model.predict(image)[0]
top3 = prediction.argsort()[::-1][:3]
top3_list = []
for i in top3:
    print(races_invert_dict[i].split('-')[1],prediction[i])
    top3_list.append(races_invert_dict[i].split('-')[1]+" = "+str(prediction[i]))

Lhasa 0.50763047
Pekinese 0.4372862
Shih 0.053308826


In [44]:
top3_list

['Lhasa = 0.50763047', 'Pekinese = 0.4372862', 'Shih = 0.053308826']